# Importing Packages and Setting Parameters

In [1]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import pandas as pd
import datetime as dt
import time
from datetime import datetime

In [2]:
# OANDA account details
access_token = "8652b0f932f8e4ae75fcbdbef4b1d695-a5d3f26be1b9d9015d0930affdee7bf3"
accountID = "101-001-25533615-002"
#api = oandapyV20.API(access_token=access_token, environment="practice")
try:
    client = oandapyV20.API(access_token=access_token, environment="practice")
except V20Error as e:
    print("Error: {}".format(e))

In [3]:
# specify the instrument to trade
instrument = "USD_CAD"

In [ ]:
# set the order parameters
order_quantity = 10000 # start with 10k units
total_order_quantity = 1000000 # the total order quantity
current_executed_quantity = 0 # keep track of how many units have been executed so far
non_executed_quantity = 0 # keep track of the units that were not executed in the previous window
average_execution_price = 0 # initialize the average execution price to 0
total_average_execution_price = 0 # initialize the total average execution price to 0

# setting the required time frames
first_execution_start_time = dt.datetime.now().replace(hour=16, minute=30, second=0, microsecond=0)
first_execution_end_time = dt.datetime.now().replace(hour=18, minute=30, second=0, microsecond=0)

second_execution_start_time = dt.datetime.now().replace(hour=19, minute=0, second=0, microsecond=0)
second_execution_end_time = dt.datetime.now().replace(hour=22, minute=0, second=0, microsecond=0)

third_execution_start_time = dt.datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
third_execution_end_time = dt.datetime.now().replace(hour=1, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

fourth_execution_start_time = dt.datetime.now().replace(hour=3, minute=0, second=0, microsecond=0) #+ dt.timedelta(days=1) # add one day to the start time
fourth_execution_end_time = dt.datetime.now().replace(hour=6, minute=0, second=0, microsecond=0) #+ dt.timedelta(days=1) # add one day to the end time

In [5]:
# define a function to execute orders
def execute_order(quantity, trades):
    # create a market order request
    mo = MarketOrderRequest(
        instrument=instrument,
        units=-quantity,
        takeProfitOnFill=None,
        stopLossOnFill=None
    )
    # send the order request
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
        })
    else:
        # Handle the case where the order is not filled
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [6]:
# make a function to get price that can be used to compare with the average price before execution takes place
def get_rate():
    price_request = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    response = client.request(price_request)
    return float(response['prices'][0]['asks'][0]['price'])

# First Interval

In [7]:
# execute the first window
while dt.datetime.now() < first_execution_start_time:
    time.sleep(1)

batch1_prices = []  # create a list to store the prices
while dt.datetime.now() >= first_execution_start_time and dt.datetime.now() <= first_execution_end_time:
    for i in range(20): # 20 iterations to execute 10k units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.2:       
            response = execute_order(order_quantity, batch1_prices)
            current_executed_quantity += order_quantity
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Successful execution for 0 th interval
Successful execution for 1 th interval
Successful execution for 2 th interval
Successful execution for 3 th interval
Successful execution for 4 th interval
Order canceled due to: MARKET_HALTED
Successful execution for 5 th interval
Successful execution for 6 th interval
Successful execution for 7 th interval
Successful execution for 8 th interval
Successful execution for 9 th interval
Successful execution for 10 th interval
Successful execution for 11 th interval
Successful execution for 12 th interval
Successful execution for 13 th interval
Successful execution for 14 th interval
Successful execution for 15 th interval
Successful execution for 16 th interval
Successful execution for 17 th interval
Successful execution for 18 th interval
Successful execution for 19 th interval


In [8]:
# convert the list of prices into a DataFrame
df1 = pd.DataFrame(batch1_prices)

In [9]:
df1

,Timestamp,Order_ID,Instrument,Price
0,2023-04-27 16:30:02.566445,218.0,USD_CAD,1.35918
1,2023-04-27 16:36:02.680576,224.0,USD_CAD,1.35926
2,2023-04-27 16:42:02.752613,230.0,USD_CAD,1.35919
3,2023-04-27 16:48:02.820869,236.0,USD_CAD,1.35920
4,2023-04-27 16:54:02.889666,242.0,USD_CAD,1.35900
5,2023-04-27 17:06:10.464852,250.0,USD_CAD,1.35891
6,2023-04-27 17:12:10.608167,258.0,USD_CAD,1.35897
7,2023-04-27 17:18:10.694996,264.0,USD_CAD,1.35896
8,2023-04-27 17:24:10.791740,268.0,USD_CAD,1.35899
9,2023-04-27 17:30:10.859788,274.0,USD_CAD,1.35907


In [10]:
average_execution_price1 = df1['Price'].sum() / len(df1)

In [11]:
average_execution_price1

1.3591310526315787

In [12]:
current_executed_quantity

200000

In [13]:
df1.to_csv("Short_Interval_1.csv", index=False)

# Second Interval

In [14]:
# execute the second window
while dt.datetime.now() < second_execution_start_time:
    time.sleep(1)

batch2_prices = []  # create a list to store the prices

while dt.datetime.now() >= second_execution_start_time and dt.datetime.now() <= second_execution_end_time:
    for i in range(30): # 30 iterations to execute 10k units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.5:
            if get_rate() <= average_execution_price1:
                execute_order(order_quantity, batch2_prices)
                current_executed_quantity += order_quantity
                
            else:
                print("Price is more than average price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Price is more than average price
Successful execution for 0 th interval
Price is more than average price
Successful execution for 1 th interval
Price is more than average price
Successful execution for 2 th interval
Price is more than average price
Successful execution for 3 th interval
Price is more than average price
Successful execution for 4 th interval
Price is more than average price
Successful execution for 5 th interval
Price is more than average price
Successful execution for 6 th interval
Price is more than average price
Successful execution for 7 th interval
Price is more than average price
Successful execution for 8 th interval
Price is more than average price
Successful execution for 9 th interval
Price is more than average price
Successful execution for 10 th interval
Price is more than average price
Successful execution for 11 th interval
Price is more than average price
Successful execution for 12 th interval
Price is more than average price
Successful execution for 13 

In [15]:
# convert the list of prices into a DataFrame
df2 = pd.DataFrame(batch2_prices)

In [16]:
df2

""


In [17]:
# concatenate the two dataframes vertically
new_df = pd.concat([df1, df2], axis=0)

In [18]:
# calculate the total average execution price
average_execution_price2 = new_df['Price'].sum() / len(new_df)

average_execution_price2

1.3591310526315787

In [19]:
current_executed_quantity

200000

In [20]:
non_executed_quantity2 = total_order_quantity * 0.3 - current_executed_quantity

non_executed_quantity2

100000.0

In [21]:
df2.to_csv("Short_Interval_2.csv", index=False)

# Third Interval

In [22]:
# execute the third window
while dt.datetime.now() < third_execution_start_time:
    time.sleep(1)

batch3_prices = []  # create a list to store the prices

while dt.datetime.now() >= third_execution_start_time and dt.datetime.now() <= third_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity2 + (total_order_quantity * 0.2)) / 20)
    for i in range(20): # 20 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.7:
            if get_rate() <= average_execution_price2:
                response = execute_order(order_quantity, batch3_prices)
                current_executed_quantity += order_quantity
            else:
                print("Price is more than average price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Price is more than average price
Successful execution for 0 th interval
Price is more than average price
Successful execution for 1 th interval
Price is more than average price
Successful execution for 2 th interval
Price is more than average price
Successful execution for 3 th interval
Price is more than average price
Successful execution for 4 th interval
Price is more than average price
Successful execution for 5 th interval
Price is more than average price
Successful execution for 6 th interval
Price is more than average price
Successful execution for 7 th interval
Price is more than average price
Successful execution for 8 th interval
Price is more than average price
Successful execution for 9 th interval
Price is more than average price
Successful execution for 10 th interval
Price is more than average price
Successful execution for 11 th interval
Price is more than average price
Successful execution for 12 th interval
Price is more than average price
Successful execution for 13 

In [23]:
# convert the list of prices into a DataFrame
df3 = pd.DataFrame(batch3_prices)

In [24]:
df3

""


In [25]:
# concatenate the two dataframes vertically
updated_df = pd.concat([new_df, df3], axis=0)

In [26]:
# calculate the total average execution price
average_execution_price3 = updated_df['Price'].sum() / len(updated_df)

average_execution_price3

1.3591310526315787

In [27]:
current_executed_quantity

200000

In [28]:
non_executed_quantity3 = total_order_quantity * 0.7 - current_executed_quantity

non_executed_quantity3

500000.0

In [30]:
df3.to_csv("Short_Interval_3.csv", index=False)

# Fourth Interval

In [32]:
# execute the fourth window
while dt.datetime.now() < fourth_execution_start_time:
    time.sleep(1)

batch4_prices = []  # create a list to store the prices

while dt.datetime.now() >= fourth_execution_start_time and dt.datetime.now() <= fourth_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity3 + (total_order_quantity * 0.3)) / 30)
    for i in range(30): # 30 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity:
            if get_rate() <= average_execution_price3:
                response = execute_order(order_quantity, batch4_prices)
                current_executed_quantity += order_quantity
            else:
                print("Price is more than average price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Price is more than average price
Successful execution for 0 th interval
Price is more than average price
Successful execution for 1 th interval
Price is more than average price
Successful execution for 2 th interval
Price is more than average price
Successful execution for 3 th interval
Price is more than average price
Successful execution for 4 th interval
Price is more than average price
Successful execution for 5 th interval
Price is more than average price
Successful execution for 6 th interval
Price is more than average price
Successful execution for 7 th interval
Price is more than average price
Successful execution for 8 th interval
Price is more than average price
Successful execution for 9 th interval
Price is more than average price
Successful execution for 10 th interval
Price is more than average price
Successful execution for 11 th interval
Price is more than average price
Successful execution for 12 th interval
Price is more than average price
Successful execution for 13 

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [33]:
# convert the list of prices into a DataFrame
df4 = pd.DataFrame(batch4_prices)

In [34]:
df4

""


In [35]:
# concatenate the two dataframes vertically
final_df = pd.concat([updated_df, df4], axis=0)

In [36]:
# calculate the total average execution price
average_execution_price4 = updated_df['Price'].sum() / len(updated_df)

average_execution_price4

1.3591310526315787

In [37]:
current_executed_quantity

200000

In [38]:
non_executed_quantity_final = total_order_quantity - current_executed_quantity

non_executed_quantity_final

800000

In [39]:
df4.to_csv("Short_Interval_4.csv", index=False)

In [40]:
final_df.to_csv("Short_Final_Executed_Data.csv", index=False)

# Generating Output CSV File

In [45]:
# Create a dictionary with your data
data = {'Interval': ['Interval 1', 'Interval 2', 'Interval 3', 'Interval 4'],
        'Executed Quantity': [200000, 0, 0, 0], 
        'Percentage of Executed Quantity': [100, 0, 0, 0],
        'Non Executed Quantity': [0, 300000, 200000, 300000],
        'Percentage of Non Executed Quantity': [0, 100, 100, 100]}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

# Add some sample data for the last three columns
df['Average Execution Price'] = [average_execution_price1, average_execution_price2, average_execution_price3, average_execution_price4]

# select columns to sum
sum_cols = ['Executed Quantity', 'Non Executed Quantity']

# select last column to calculate average
avg_col = 'Average Execution Price'

# calculate the total and average values
total = df[sum_cols].sum()
average = df[avg_col].mean()

# create a new row with the calculated values
new_row = pd.DataFrame({"Interval": "Total Vectors",
                        'Executed Quantity': [total['Executed Quantity']], 
                        'Percentage of Executed Quantity': int(total['Executed Quantity'] / 1000000 * 100),
                        'Non Executed Quantity': [total['Non Executed Quantity']],
                        'Percentage of Non Executed Quantity': int(total['Non Executed Quantity'] / 1000000 * 100),
                        'Average Execution Price': [average] })

# concatenate the original DataFrame with the new row
df = pd.concat([df, new_row])

# reset the index
df = df.reset_index(drop=True)

# Convert the DataFrame to a CSV file
df.to_csv('Short - Execution Output Vectors.csv', index=False)